In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {cricketer}")

output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

chain.invoke({"cricketer": "Rohit Sharma"})

"Here's one:\n\nRohit Sharma is the only player in international cricket to have scored three double centuries in One-Day Internationals (ODIs). He achieved this feat in 2013, 2014, and 2017, respectively. What's even more remarkable is that all three of his double centuries have come against different teams: Australia, Sri Lanka, and Sri Lanka again!"

## Use of .bind() to add arguments to a Runnable in a LCEL Chain.

* For example: we can add an argument to stop the model response when it reaches the word "Rohit Sharma".

In [8]:
chain = prompt | model.bind(stop=["Sharma"]) | output_parser

chain.invoke({"cricketer": "Rohit Sharma"})

"Here's a curious fact about Rohit "

## Combining LCEL Chains

In [9]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [20]:
chain.invoke("Lincoln")

'Abraham Lincoln, the 16th President of the United States, issued the Emancipation Proclamation in 1863, declaring freedom for all slaves in Confederate territory.'

### Combine chain

In [24]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

combine_chain = {"politician": chain} | historian_prompt | model | output_parser

In [25]:
combine_chain.invoke({"politician": "Lincoln"})

"What a great question!\n\nAbsolutely, Abraham Lincoln's Emancipation Proclamation in 1863 was a profoundly positive step for humanity. Here's why:\n\n1. **End of Slavery**: The Emancipation Proclamation paved the way for the eventual abolition of slavery in the United States. It declared that all slaves in Confederate territory were free, effective on January 1, 1863. This was a crucial step towards ending the institution of slavery, which was a moral evil that had plagued the country since its founding.\n2. **Advancement of Human Rights**: The Emancipation Proclamation recognized the inherent dignity and humanity of enslaved people, acknowledging that they were entitled to freedom and equality. This marked a significant milestone in the struggle for human rights and social justice in the United States.\n3. **Unity and Equality**: By issuing the Emancipation Proclamation, Lincoln took a crucial step towards uniting the country and promoting equality among all citizens. He emphasized t

## Another Example: A Chain inside another chain.

In [35]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

prompt1 = ChatPromptTemplate.from_template("What is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template("What continent is the country {country} in? respond in {language}")

output_parser = StrOutputParser()

model = ChatGroq(model="llama3-70b-8192")

chain1 = prompt1 | model | output_parser

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | output_parser
)

chain2.invoke({"politician": "Miterrand", "language": "English"})

'François Mitterrand was the President of France from 1981 to 1995, so the country he is from is France. France is located in the continent of Europe.'

In [37]:
chain2.invoke({"politician": "Miterrand", "language": "Spanish"})

'Excelente deducción! François Mitterrand fue el presidente de Francia desde 1981 hasta 1995. Por lo tanto, el país del que es originario es Francia, que se encuentra en el continente de Europa.'

In [38]:
chain2.invoke({"politician": "Miterrand", "language": "Hindi"})

'आप शायद फ्रांस के राष्ट्रपति फ्रांस्वा मिटरांड के बारे में सोच रहे हैं, जिन्होंने 1981 से 1995 तक फ्रांस के राष्ट्रपति के रूप में कार्य किया। इसलिए, मिटरांड से जुड़ा देश फ्रांस है! फ्रांस यूरोप महाद्वीप में स्थित है।'

## LCEL Chain at work in typical RAG Application

In [44]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")

In [46]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap= 200)

splits = text_splitter.split_documents(docs)

vector_db = Chroma.from_documents(documents=splits, embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'}))

retriever = vector_db.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


c:\Users\Lokesh\anaconda3\envs\llmapp\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [48]:
rag_chain.invoke("what is task decomposition?")

c:\Users\Lokesh\anaconda3\envs\llmapp\Lib\site-packages\pydantic\main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This can be done using various techniques, such as Chain of Thought (CoT) or Tree of Thoughts, which involve "thinking step by step" to decompose hard tasks into simpler ones. The goal is to transform big tasks into multiple manageable tasks, making it easier to plan and execute them.'